# Conference scheduling problem

In [29]:
# !pip install mip

In [1]:
import pandas as pd
import numpy as np

## Problem Statement

Recently Slalom Seattle was planning it's annual Innovation Symposium, and I was on the planning committee.

The planning team wanted to assign each individual to a schedule. Attendees had been asked to rank their preference for attending each topic. The problem was a large number of schedules would have to manually be created; over 600 attendees, needed to be assigned to 7 different topics across 3 sessions. The original plan was to whiteboard it and manually create schedules and assign people to schedules, an excruciating process that may have to be rerun as new responses or constraints came in. It also seemed difficult to ensure the best solution: how do you even pick the best subset of possible schedules?

There were also some constraints about the capacity of available rooms, more popular sessions could be put in a larger room, and we wanted to avoid having rooms half empty in one session and then bursting at the seams in a later session.

### 2022 Bonus Complexity
In 2022 a number of the attendees will be virtual while

**Problem:** Creates quadratic complexity when adding in in person / virtual constraint. Maybe there's a better way to solve this?  
Have tried converting bool quadratic into 4 constraints (for each multiplication!!!!).  
https://orinanobworld.blogspot.com/2010/10/binary-variables-and-quadratic-terms.html  
https://python-mip.readthedocs.io/en/latest/examples.html  
https://docs.python-mip.com/en/latest/examples.html  

## Data Prep

### Let's have a look at the survey data

In [131]:
# Read data
file='data/selections_2022.csv'
df = pd.read_csv(file)
df.head(5)

,Do you plan on attending the Shape the Future Symposium?,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5,Choice 6,Choice 7,Choice 8,Choice 9,Choice 10,Choice 11,Choice 12,Choice 13
0,"Yes, I will attend in person",Architecting for a Sustainable Cloud Future,The Future of Work: What You Need to Know to N...,Environmental Impact @ Slalom with Net Zero Cloud,Starbucks Experimentation Program,Shaping the Future Through Inspiration and Inn...,ESG 101,Introduction to the Seattle Sustainability Team,Energy Demand Nowcasting: How Machine Learning...,Modern Portfolio Management,Build a Bot Using PowerAutomate,Fusion Development with the Microsoft Power Pl...,Low Code – Meeting User Needs While Also Being...,MIDL Earth - Digitized
1,"Yes, I will attend virtually",Fusion Development with the Microsoft Power Pl...,Architecting for a Sustainable Cloud Future,Service Design: A Framework for Practical Coll...,Build a Bot Using PowerAutomate,Shaping the Future Through Inspiration and Inn...,Introduction to the Seattle Sustainability Team,ESG 101,Low Code – Meeting User Needs While Also Being...,Energy Demand Nowcasting: How Machine Learning...,The Future of Work: What You Need to Know to N...,Environmental Impact @ Slalom with Net Zero Cloud,MIDL Earth - Digitized,Starbucks Experimentation Program
2,"Yes, I will attend virtually",Build a Bot Using PowerAutomate,Architecting for a Sustainable Cloud Future,Low Code – Meeting User Needs While Also Being...,MIDL Earth - Digitized,Fusion Development with the Microsoft Power Pl...,Shaping the Future Through Inspiration and Inn...,Introduction to the Seattle Sustainability Team,ESG 101,Energy Demand Nowcasting: How Machine Learning...,The Future of Work: What You Need to Know to N...,Environmental Impact @ Slalom with Net Zero Cloud,Starbucks Experimentation Program,Modern Portfolio Management
3,"Yes, I will attend in person",Build a Bot Using PowerAutomate,Fusion Development with the Microsoft Power Pl...,Shaping the Future Through Inspiration and Inn...,Introduction to the Seattle Sustainability Team,ESG 101,Low Code – Meeting User Needs While Also Being...,Architecting for a Sustainable Cloud Future,Energy Demand Nowcasting: How Machine Learning...,The Future of Work: What You Need to Know to N...,Environmental Impact @ Slalom with Net Zero Cloud,MIDL Earth - Digitized,Starbucks Experimentation Program,Modern Portfolio Management
4,"Yes, I will attend in person",Starbucks Experimentation Program,The Future of Work: What You Need to Know to N...,Low Code – Meeting User Needs While Also Being...,Build a Bot Using PowerAutomate,Fusion Development with the Microsoft Power Pl...,Shaping the Future Through Inspiration and Inn...,Introduction to the Seattle Sustainability Team,ESG 101,Architecting for a Sustainable Cloud Future,Energy Demand Nowcasting: How Machine Learning...,Environmental Impact @ Slalom with Net Zero Cloud,MIDL Earth - Digitized,Modern Portfolio Management


In [133]:
# Drop non attendees
df = df[df['Do you plan on attending the Shape the Future Symposium?'] != 'No, I will not attend']

In [134]:
a_virt = df[['Do you plan on attending the Shape the Future Symposium?']].copy()
df.drop('Do you plan on attending the Shape the Future Symposium?', axis=1, inplace=True)
# df = df[df.columns[:7]]

In [135]:
a_virt.value_counts()

Do you plan on attending the Shape the Future Symposium?
Yes, I will attend in person                                279
Yes, I will attend virtually                                148
dtype: int64

In [141]:
a_virt = (a_virt == 'Yes, I will attend virtually').values.ravel()

Each row represents an attendee, and the columns are their ranked preference for each topic. Choice 1 was their highest preference and Choice 7 was their lowest.

In [142]:
df.describe()

,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5,Choice 6,Choice 7,Choice 8,Choice 9,Choice 10,Choice 11,Choice 12,Choice 13
count,415,415,415,415,415,415,415,415,415,415,415,415,415
unique,15,15,15,15,15,15,15,15,14,14,15,15,15
top,Build a Bot Using PowerAutomate,The Future of Work: What You Need to Know to N...,Service Design: A Framework for Practical Coll...,Shaping the Future Through Inspiration and Inn...,Build a Bot Using PowerAutomate,Fusion Development with the Microsoft Power Pl...,Introduction to the Seattle Sustainability Team,ESG 101,Architecting for a Sustainable Cloud Future,Energy Demand Nowcasting: How Machine Learning...,Environmental Impact @ Slalom with Net Zero Cloud,MIDL Earth - Digitized,Modern Portfolio Management
freq,67,53,46,47,116,104,78,80,77,92,95,110,105


Looks like there are a number of missing values where someone either hasn't provided a ranking (but they've confirmed they're attending) or they have only provided a top 1,2,3... We'll have to handle all of these cases.

In [143]:
# Get total number of respondants
N_total = len(df)
print(f'Total number of responses: {N_total}')

# Create a dictionary of the topics for later use
topics = np.unique(df.dropna().values)
topic_map = dict(zip(topics,range(len(topics))))

Total number of responses: 427


In [145]:
topics

array(['Architecting for a Sustainable Cloud Future',
       'Build a Bot Using PowerAutomate',
       'Customer Data Platforms:  The Innovative intersection of Martech, Machine Learning, and Customer Data',
       'ESG 101',
       'Energy Demand Nowcasting: How Machine Learning can Help Unlock ESG Enablement',
       'Environmental Impact @ Slalom with Net Zero Cloud',
       'Fusion Development with the Microsoft Power Platform',
       'Introduction to the Seattle Sustainability Team',
       'Low Code – Meeting User Needs While Also Being Fast and Lean',
       'MIDL Earth - Digitized ', 'Modern Portfolio Management',
       'Service Design: A Framework for Practical Collaboration',
       'Shaping the Future Through Inspiration and Innovation',
       'Starbucks Experimentation Program',
       'The Future of Work: What You Need to Know to Navigate this Brave New World'],
      dtype=object)

### Convert the data into a convenient format

The data format isn't ideal; we'd like to have topics as the columns and rankings as integer values in the table.

Let's convert this into a more convenient shape.

In [146]:
# Convert to ranked format
df = df.reset_index().melt(id_vars='index').dropna().pivot(columns='value',index='index', values='variable') # Index = the id of a particular response
df = df.reindex(pd.RangeIndex(0,N_total)) # Reindex becuase the previous step dropped all the rows where respondant didn't provide any preferences at all

In [148]:
df.iloc[0]

value
Architecting for a Sustainable Cloud Future                                                               Choice 1
Build a Bot Using PowerAutomate                                                                          Choice 10
Customer Data Platforms:  The Innovative intersection of Martech, Machine Learning, and Customer Data          NaN
ESG 101                                                                                                   Choice 6
Energy Demand Nowcasting: How Machine Learning can Help Unlock ESG Enablement                             Choice 8
Environmental Impact @ Slalom with Net Zero Cloud                                                         Choice 3
Fusion Development with the Microsoft Power Platform                                                     Choice 11
Introduction to the Seattle Sustainability Team                                                           Choice 7
Low Code – Meeting User Needs While Also Being Fast and Lean              

In [149]:
# Convert choice rankings to integers
def choice_to_int(choice):
    try:
        return int(choice.split(' ')[-1])
    except:
        return np.nan

choice_to_int('Choice 2'), choice_to_int(np.nan)

(2, nan)

In [150]:
# Use applymap to clean every choice
df = df.applymap(choice_to_int)

In [151]:
df.iloc[0]

value
Architecting for a Sustainable Cloud Future                                                               1.0
Build a Bot Using PowerAutomate                                                                          10.0
Customer Data Platforms:  The Innovative intersection of Martech, Machine Learning, and Customer Data     NaN
ESG 101                                                                                                   6.0
Energy Demand Nowcasting: How Machine Learning can Help Unlock ESG Enablement                             8.0
Environmental Impact @ Slalom with Net Zero Cloud                                                         3.0
Fusion Development with the Microsoft Power Platform                                                     11.0
Introduction to the Seattle Sustainability Team                                                           7.0
Low Code – Meeting User Needs While Also Being Fast and Lean                                             12.0
MIDL

We also want to fill missing preferences; we'll assume an individual's preference for topics not ranked is uniform but lower than those they have ranked.

In [152]:
# Add one to the max choice for each row
unranked_preference = df.max(axis=1).fillna(0) +1

# The column by column replace the missing values with the max per row
for col in df.columns:
    df[col] = df[col].fillna(value=unranked_preference)
df.sample(5, random_state=1990)

value,Architecting for a Sustainable Cloud Future,Build a Bot Using PowerAutomate,"Customer Data Platforms: The Innovative intersection of Martech, Machine Learning, and Customer Data",ESG 101,Energy Demand Nowcasting: How Machine Learning can Help Unlock ESG Enablement,Environmental Impact @ Slalom with Net Zero Cloud,Fusion Development with the Microsoft Power Platform,Introduction to the Seattle Sustainability Team,Low Code – Meeting User Needs While Also Being Fast and Lean,MIDL Earth - Digitized,Modern Portfolio Management,Service Design: A Framework for Practical Collaboration,Shaping the Future Through Inspiration and Innovation,Starbucks Experimentation Program,The Future of Work: What You Need to Know to Navigate this Brave New World
218,5.0,6.0,14.0,8.0,4.0,11.0,3.0,12.0,2.0,13.0,10.0,9.0,7.0,14.0,1.0
272,10.0,1.0,14.0,9.0,11.0,3.0,8.0,7.0,5.0,12.0,6.0,14.0,4.0,13.0,2.0
99,7.0,1.0,14.0,5.0,8.0,10.0,2.0,4.0,6.0,11.0,13.0,14.0,3.0,12.0,9.0
264,9.0,6.0,14.0,8.0,10.0,3.0,7.0,2.0,5.0,12.0,13.0,14.0,1.0,4.0,11.0
257,11.0,1.0,14.0,9.0,6.0,12.0,5.0,8.0,10.0,13.0,14.0,7.0,4.0,3.0,2.0


You can see the response at index `410` didn't provide any choice so every topic has a rank 1.

## Solve Using Linear Programming

We can restate this as a linear programming problem where we have various constraints on the number of attendees in each session, and the number of sessions and topics a particular person can attend. 

### Assign preference cost
We want to assign costs for assigning a particular person to a session, that way we can compute a total of cost of a configuration, and optimize. We can say the cost is zero for assigning them to one of their top 3 sessions, with an increasing cost if we assign them to a lower preference session.

Depending on the problem and constraints we might choose different values here. For example if we want to avoid assigning any non-top 3 choices at all we could use a higher value for Choice 4 and on.

In [154]:
cost_array = [0,0,0]
cost_remap = {i:cost_array[i-1] for i in range(1,len(cost_array)+1)}
print(f'Costs by choice: {cost_remap}')

# Remap values in dataframe
df = df.replace(cost_remap).astype(np.int8) # apply our map, and convert to int
df.sample(5, random_state=1990)

Costs by choice: {1: 0, 2: 0, 3: 0}


value,Architecting for a Sustainable Cloud Future,Build a Bot Using PowerAutomate,"Customer Data Platforms: The Innovative intersection of Martech, Machine Learning, and Customer Data",ESG 101,Energy Demand Nowcasting: How Machine Learning can Help Unlock ESG Enablement,Environmental Impact @ Slalom with Net Zero Cloud,Fusion Development with the Microsoft Power Platform,Introduction to the Seattle Sustainability Team,Low Code – Meeting User Needs While Also Being Fast and Lean,MIDL Earth - Digitized,Modern Portfolio Management,Service Design: A Framework for Practical Collaboration,Shaping the Future Through Inspiration and Innovation,Starbucks Experimentation Program,The Future of Work: What You Need to Know to Navigate this Brave New World
218,0,4,14,6,0,11,0,12,0,13,10,9,5,14,0
272,10,0,14,9,11,0,6,5,0,12,4,14,0,13,0
99,5,0,14,0,6,10,0,0,4,11,13,14,0,12,9
264,9,4,14,6,10,0,5,0,0,12,13,14,0,0,11
257,11,0,14,9,4,12,0,6,10,13,14,5,0,0,0


In [155]:
df.iloc[0]

value
Architecting for a Sustainable Cloud Future                                                               0
Build a Bot Using PowerAutomate                                                                          10
Customer Data Platforms:  The Innovative intersection of Martech, Machine Learning, and Customer Data    14
ESG 101                                                                                                   4
Energy Demand Nowcasting: How Machine Learning can Help Unlock ESG Enablement                             6
Environmental Impact @ Slalom with Net Zero Cloud                                                         0
Fusion Development with the Microsoft Power Platform                                                     11
Introduction to the Seattle Sustainability Team                                                           5
Low Code – Meeting User Needs While Also Being Fast and Lean                                             12
MIDL Earth - Digitized

The response at index `410` which didn't provide any preferences has a zero cost; it doesn't matter which session they are assigned to. 

### Convert to a 3D cost matrix
We need to assign a cost for allocating each attendee to a topic in each session. There are three sessions and the cost should be the same for a particular attendee to attend a topic regardless of the order they attend. So we need to transform our 2D array of costs into a 3D matrix.

In [156]:
cost_matrix = np.dot(np.expand_dims(df.values,-1), np.array([[1,1,1]]))
print(f'First dataframe row:\n{df.iloc[0]}', f'Corresponding item from cost array:\n{cost_matrix[0]}', f'Shape of full array: {cost_matrix.shape}', sep='\n\n')

First dataframe row:
value
Architecting for a Sustainable Cloud Future                                                               0
Build a Bot Using PowerAutomate                                                                          10
Customer Data Platforms:  The Innovative intersection of Martech, Machine Learning, and Customer Data    14
ESG 101                                                                                                   4
Energy Demand Nowcasting: How Machine Learning can Help Unlock ESG Enablement                             6
Environmental Impact @ Slalom with Net Zero Cloud                                                         0
Fusion Development with the Microsoft Power Platform                                                     11
Introduction to the Seattle Sustainability Team                                                           5
Low Code – Meeting User Needs While Also Being Fast and Lean                                             12
M

The resulting array has a shape (`Number of respondants`, `Number of Topics`, `Number of Sessions`) = (`578`,`7`,`3`). So we have 12,138‬ constraints to solve... that's a bit more than [Excel's Solver package](https://www.excel-easy.com/data-analysis/solver.html) limit of 200.

### Mixed Integer Programming from Google's Operations Reaserch package to the rescue

Google provides a fast and open source library for solving various combinatorial optimization problems: [Google OR tools](https://developers.google.com/optimization/)([GitHub](https://github.com/google/or-tools)). There are a few ways we could solve this but we'll use the [Mixed Integer Programming (MIP) package](https://developers.google.com/optimization/assignment/assignment_mip).

First we need to provide a room capacity, we can adjust these numbers iteratively to best fit sessions into rooms. The iteration process went as follows: assign room capacities run solver, evaluate variance in attendance across sessions and adjust session capacities accordingly.

In [157]:
len(topics)

15

In [159]:
# Set max capacity per room
topic_capacities = [40]*len(topics)
list(zip(topics,topic_capacities))

[('Architecting for a Sustainable Cloud Future', 40),
 ('Build a Bot Using PowerAutomate', 40),
 ('Customer Data Platforms:  The Innovative intersection of Martech, Machine Learning, and Customer Data',
  40),
 ('ESG 101', 40),
 ('Energy Demand Nowcasting: How Machine Learning can Help Unlock ESG Enablement',
  40),
 ('Environmental Impact @ Slalom with Net Zero Cloud', 40),
 ('Fusion Development with the Microsoft Power Platform', 40),
 ('Introduction to the Seattle Sustainability Team', 40),
 ('Low Code – Meeting User Needs While Also Being Fast and Lean', 40),
 ('MIDL Earth - Digitized ', 40),
 ('Modern Portfolio Management', 40),
 ('Service Design: A Framework for Practical Collaboration', 40),
 ('Shaping the Future Through Inspiration and Innovation', 40),
 ('Starbucks Experimentation Program', 40),
 ('The Future of Work: What You Need to Know to Navigate this Brave New World',
  40)]

In [160]:
N_topics = len(topics)
N_sessions = 3

### Create a model and define contraints

In [318]:
a_virt = [bool(x) for x in a_virt]

In [187]:
a_in_p = [not bool(x) for x in a_virt]

In [299]:
not a_in_p[0]

False

In [301]:
## Create solver model
from mip import Model, xsum, BINARY, minimize, OptimizationStatus

model = Model()

## Create assignment array
# A boolean (True/False) value for each possible attendee, topic, session combination
# We'll store these in a Numpy array of object datatype for easy indexing
# By summing across dimesions of the array we can view and enforce constraints
# In Python True is evaluated as 1 in a summation and False is evaluated as 0
x = np.zeros((N_total,N_topics,N_sessions), dtype=object)
for attendee in range(N_total):
    for topic in range(N_topics):
        for session in range(N_sessions):
            x[attendee, topic, session] = model.add_var(f'x[{attendee}, {topic}, {session}]',
                                                        var_type=BINARY)

# Variable for virtual sessions            
s_virt = np.zeros((N_topics,N_sessions), dtype=object)
for t in range(N_topics):
    for s in range (N_sessions):
        s_virt[t,s] = model.add_var(f's_v[{t},{s}]', var_type=BINARY)

# allow multiplication of variables as per https://groups.google.com/g/python-mip/c/rhS_a78_qx8
z = np.zeros((N_total,N_topics,N_sessions), dtype=object)
for a in range(N_total):
    for t in range(N_topics):
        for s in range (N_sessions):
            z[a,t,s] = model.add_var(f'z[{a},{t},{s}]', var_type=BINARY)
            model.add_constr( z[a,t,s] <= x[a,t,s] )
            model.add_constr( z[a,t,s] <= s_virt[t,s] )
            model.add_constr( z[a,t,s] >= x[a,t,s] + s_virt[t,s] - 1 )

# Ensure there are at least some virtual sessions
model += xsum([s_virt[t,s] for t in range(N_topics) for s in range(N_sessions)]) >= 14
        
obj = []
for a in range(N_total):
    for t in range(N_topics):
        for s in range (N_sessions):
            obj.append(cost_matrix[a,t,s] * x[a,t,s])
            obj.append(9*z[a,t,s]) # minimize the number of in person people attending virtual sessions and vice versa
            
model.objective = minimize(xsum(obj))

## Add constraints to the model
# Each attendee attends exactly one topic per session
# Therefore the sum across topics for each attendee, session is 1
for a in range(N_total):
    for s in range(N_sessions):
        model += xsum(x[a,t,s] for t in range(N_topics)) == 1

# Each attendee attends each topic no more than 1 time
# For each attendee, topic the sum can be 1 or 0
for a in range(N_total):
    for t in range(N_topics):
        model += xsum(x[a,t,s] for s in range(N_sessions)) <= 1

# Max people per session
# Total count of attendees for each session to be less than capacities defined
for s in range(N_sessions):
    for t in range(N_topics):
        model += xsum(x[a,t,s] for a in range(N_total)) <= topic_capacities[t]

In [302]:
# optimizing
model.optimize(max_seconds = 30*60)

<OptimizationStatus.FEASIBLE: 3>

In [310]:
model.objective.x

507.0

In [303]:
s_sol = np.full_like(s_virt,0, dtype=int)
for t in range(N_topics):
    for s in range(N_sessions):
        s_sol[t,s] = s_virt[t,s].x

In [304]:
# Return resulting solution to a new array
x_sol = np.full_like(x, 0, dtype=int)
for a in range(N_total):
    for t in range(N_topics):
        for s in range(3):
            x_sol[a,t,s] = x[a,t,s].x

In [305]:
# Return resulting solution to a new array
z_sol = np.full_like(z, 0, dtype=int)
for a in range(N_total):
    for t in range(N_topics):
        for s in range(3):
            z_sol[a,t,s] = z[a,t,s].x

In [306]:
# People not in top 3
np.sum((x_sol * cost_matrix))

21

In [307]:
not_top_3 = np.where(x_sol*cost_matrix>0)
for i,a in enumerate(not_top_3[0]):
    print(f'Attendee {a} for topic {not_top_3[1][i]}, session {not_top_3[2][i]}, had a cost of {cost_matrix[a,not_top_3[1][i],not_top_3[2][i]]}', end='\n')

Attendee 3 for topic 0, session 1, had a cost of 5
Attendee 11 for topic 0, session 1, had a cost of 10
Attendee 373 for topic 6, session 2, had a cost of 6


In [308]:
# Number of people in person that are attending a virtual only event
in_person_attending_virtual = np.array([a * s_sol for a in a_in_p]) * x_sol
np.sum(in_person_attending_virtual)

41

In [313]:
ipav

(array([ 13,  14,  16,  25,  28,  31,  70, 121, 136, 137, 141, 144, 145,
        152, 154, 157, 158, 163, 163, 167, 176, 185, 201, 235, 240, 243,
        253, 256, 264, 269, 275, 305, 320, 326, 341, 342, 346, 352, 359,
        364, 384], dtype=int64),
 array([7, 7, 7, 7, 7, 9, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 9, 7, 7, 9,
        7, 9, 7, 7, 9, 7, 7, 7, 5, 7, 9, 7, 9, 7, 7, 9, 9, 7, 7],
       dtype=int64),
 array([1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0,
        0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1],
       dtype=int64))

In [317]:
ipav = np.where(in_person_attending_virtual > 0)
for i,a in enumerate(ipav[0]):
    print(f'Attendee {a} for topic {ipav[1][i]}, session {ipav[2][i]}, is virtual: {a_virt[a]}. Session is virtual: {s_sol[ipav[1][i],ipav[2][i]]==1}', end='\n')

Attendee 13 for topic 7, session 1, is virtual: False. Session is virtual: True
Attendee 14 for topic 7, session 1, is virtual: False. Session is virtual: True
Attendee 16 for topic 7, session 0, is virtual: False. Session is virtual: True
Attendee 25 for topic 7, session 1, is virtual: False. Session is virtual: True
Attendee 28 for topic 7, session 0, is virtual: False. Session is virtual: True
Attendee 31 for topic 9, session 0, is virtual: False. Session is virtual: True
Attendee 70 for topic 7, session 0, is virtual: False. Session is virtual: True
Attendee 121 for topic 7, session 1, is virtual: False. Session is virtual: True
Attendee 136 for topic 7, session 0, is virtual: False. Session is virtual: True
Attendee 137 for topic 7, session 0, is virtual: False. Session is virtual: True
Attendee 141 for topic 7, session 1, is virtual: False. Session is virtual: True
Attendee 144 for topic 7, session 0, is virtual: False. Session is virtual: True
Attendee 145 for topic 7, session 0

In [309]:
# Number of virtual attending an in person event
virtual_attending_in_person = np.array([a * 1 * (s_sol == 0) for a in a_virt]) * x_sol
np.sum(virtual_attending_in_person)

431

In [311]:
indexes_of_virtual_attending_ip = np.unique(np.where(virtual_attending_in_person > 0)[0])
ivap = np.where(virtual_attending_in_person > 0)
for i,a in enumerate(ivap[0]):
    print(f'Attendee {a} for topic {ivap[1][i]}, session {ivap[2][i]}, is virtual: {a_virt[a]}. Session is virtual: {s_sol[ivap[1][i],ivap[2][i]]==1}', end='\n')

Attendee 1 for topic 0, session 0, is virtual: True. Session is virtual: False
Attendee 1 for topic 6, session 2, is virtual: True. Session is virtual: False
Attendee 1 for topic 11, session 1, is virtual: True. Session is virtual: False
Attendee 2 for topic 0, session 2, is virtual: True. Session is virtual: False
Attendee 2 for topic 8, session 1, is virtual: True. Session is virtual: False
Attendee 7 for topic 0, session 1, is virtual: True. Session is virtual: False
Attendee 7 for topic 8, session 0, is virtual: True. Session is virtual: False
Attendee 7 for topic 12, session 2, is virtual: True. Session is virtual: False
Attendee 9 for topic 4, session 1, is virtual: True. Session is virtual: False
Attendee 9 for topic 5, session 0, is virtual: True. Session is virtual: False
Attendee 9 for topic 13, session 2, is virtual: True. Session is virtual: False
Attendee 10 for topic 1, session 1, is virtual: True. Session is virtual: False
Attendee 10 for topic 3, session 0, is virtual: 

## Evaluating the Solution

### Did our solver find an optimal solution?

In [174]:
model.status == OptimizationStatus.OPTIMAL

True

In this case it did, great! If we tighten the constraints this might not always be true. For example reducing the room capacities.

What about the total cost:

In [175]:
model.objective.x

-15525.0

Great this means everyone gets to attend their top 3 sessions.

### How many attendees in each session?
We want to check that there isn't significant varaince across sessions so that we avoid half empty rooms in some sessions.

In [176]:
print("Total numbers in each session")
pd.DataFrame(x_sol.sum(axis=0),index=topics,columns=['S1','S2','S3'], dtype=int)

Total numbers in each session


,S1,S2,S3
Architecting for a Sustainable Cloud Future,32,19,40
Build a Bot Using PowerAutomate,40,40,32
"Customer Data Platforms: The Innovative intersection of Martech, Machine Learning, and Customer Data",28,10,15
ESG 101,40,40,19
Energy Demand Nowcasting: How Machine Learning can Help Unlock ESG Enablement,37,24,15
Environmental Impact @ Slalom with Net Zero Cloud,40,39,7
Fusion Development with the Microsoft Power Platform,40,11,40
Introduction to the Seattle Sustainability Team,8,30,32
Low Code – Meeting User Needs While Also Being Fast and Lean,39,16,40
MIDL Earth - Digitized,6,40,21


## Save resulting conference schedules

In [119]:
schedule = np.argmax(x_sol,axis=1)
schedule_text = topics[schedule]
sched_df = pd.DataFrame(schedule_text, columns=[f'Session {i}' for i in range(1,4)])
sched_df.sample(5, random_state=1990)

,Session 1,Session 2,Session 3
410,Modern Culture of Data,Design Thinking for Experience & Service Design,Internet of Things
50,[Design Thinking] + [Lean] + [Agile],Modern Culture of Data,Design Thinking for Experience & Service Design
146,Design Thinking for Experience & Service Design,Modern Culture of Data,Cloud Enablement 201
315,Internet of Things,[Design Thinking] + [Lean] + [Agile],Design Thinking for Experience & Service Design
576,Modern Culture of Data,Design Thinking for Experience & Service Design,[Design Thinking] + [Lean] + [Agile]


In [19]:
sched_df.to_csv('data/conference_schedule.csv',index=True)